# Kütüphane Yükleme işlemleri
Kernel olarak oluşturduğumuz venv seçilmeli

In [ ]:
!pip install langchain_google_genai streamlit google-generativeai google-genai scikit-learn numpy python-dotenv pandas chromadb --quiet
!pip install -qU "langchain[google-genai]"



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# LM STUDIO - Embedding Model erişimleri

In [9]:
import requests
import json

# Yerel sunucunuzun adresi
API_URL = "http://localhost:1234/v1/models"

try:
    # GET isteğini gönder
    # /v1/models endpoint'i genellikle GET metodu bekler ve veri (payload) göndermez
    response = requests.get(API_URL)
    response.raise_for_status()  # HTTP hatalarını (4xx, 5xx) yakalamak için

    # Yanıtı JSON formatında al
    data = response.json()

    print("✅ Modeller Listesi İsteği Başarılı.")
    print("-" * 30)
    
    # Modeller listesini ekrana yazdır
    if 'data' in data and data['data']:
        print(f"Toplam Yüklü Model: {len(data['data'])}")
        print("\nYüklü Modeller:")
        
        for model in data['data']:
            # LM Studio'dan gelen yanıtta modelin ID'sini ve potansiyel olarak diğer bilgilerini yazdır
            model_id = model.get('id', 'Bilinmiyor')
            model_type = model.get('object', 'Bilinmiyor')
            
            # Bazı sunucular modelin durumunu (örneğin, kullanımda olup olmadığını) da döndürebilir
            print(f"- ID: {model_id} (Tip: {model_type})")

    else:
        print("UYARI: Sunucu modeller listesini boş döndürdü veya format beklenenden farklı.")
        print("Tam Yanıt:", data)

except requests.exceptions.RequestException as e:
    print(f"❌ İstek Hatası oluştu: Sunucuya ulaşılamadı veya bağlantı sorunu var.")
    print(f"Hata detayı: {e}")
except Exception as e:
    print(f"❌ Beklenmeyen bir hata oluştu: {e}")

✅ Modeller Listesi İsteği Başarılı.
------------------------------
Toplam Yüklü Model: 2

Yüklü Modeller:
- ID: embeddinggemma-300m-qat (Tip: model)
- ID: text-embedding-nomic-embed-text-v1.5 (Tip: model)


In [10]:
# Yerel sunucunuzun adresi
API_URL = "http://localhost:1234/v1/embeddings"

# Curl komutundaki JSON gövdesi
payload = {
    "model": "text-embedding-nomic-embed-text-v1.5",
    "input": "What day is it today?"
}

# Başlıklar
headers = {
    "Content-Type": "application/json"
}

try:
    response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
    response.raise_for_status()

    data = response.json()

    if data and 'data' in data and data['data']:
        embedding_vector = data['data'][0]['embedding']

        print("✅ Embedding İsteği Başarılı (POST metodu kullanıldı).")
        print(f"Embedding Boyutu: {len(embedding_vector)}")

    else:
        print("UYARI: API yanıtı beklenenden farklı veya boş.")

except requests.exceptions.RequestException as e:
    print(f"❌ İstek Hatası oluştu: {e}")

✅ Embedding İsteği Başarılı (POST metodu kullanıldı).
Embedding Boyutu: 768


In [15]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
gemini=ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)
#response=gemini.invoke("Kahve nedir?")
#print(response.content)


Kahve, kavrulmuş kahve çekirdeklerinden demlenerek hazırlanan, genellikle sıcak tüketilen ve uyarıcı etkisi olan popüler bir içecektir. Dünya genelinde yaygın olarak tüketilir ve farklı hazırlama yöntemleri, aromalar ve tatlar sunar.

**Temel Özellikleri:**

*   **Köken:** Kahve çekirdekleri, Coffea cinsine ait bitkilerin meyvelerinden elde edilir.
*   **Kavurma:** Yeşil kahve çekirdekleri, aroma ve lezzetlerini ortaya çıkarmak için kavrulur. Kavurma derecesi, kahvenin tadını ve aromasını önemli ölçüde etkiler.
*   **Demleme:** Kavrulmuş çekirdekler öğütülerek suyla demlenir. Demleme yöntemleri arasında filtre kahve, espresso, Türk kahvesi, French press gibi çeşitli seçenekler bulunur.
*   **İçerik:** Kahve, kafein, antioksidanlar ve diğer bileşikler içerir.
*   **Etkileri:** Kafein içeriği sayesinde uyarıcı etkiye sahiptir, uyanıklığı artırır ve zihinsel performansı iyileştirebilir.
*   **Çeşitlilik:** Dünya genelinde farklı kahve türleri (Arabica, Robusta, Liberica, Excelsa) ve yöres

In [12]:
import glob
import os
from google import genai
from google.genai import types
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. API Anahtarını Ayarlama
# Ortam değişkeninden anahtarı alacaktır.
try:
    client = genai.Client()
except Exception as e:
    print(f"Hata: Gemini API anahtarı ayarlanmamış veya geçersiz. Lütfen 'export GEMINI_API_KEY=\"...\"' komutunu kullanın. Detay: {e}")
    exit()

# 2. Veri Seti (Knowledge Base) Hazırlama
# Bu basit örnekte, veri setimiz bir Python listesi içindeki metin parçalarıdır.
KNOWLEDGE_BASE = []  # Boş bir listeyle başla
data_files_path = "data/*.txt"

print("Dosyalar okunuyor ve filtreleniyor...")

# 'data' klasöründeki tüm .txt dosyalarını döngüye al
for file_path in glob.glob(data_files_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            
            # strip() komutu başındaki/sonundaki boşlukları siler.
            # Eğer geriye bir şey kalırsa (yani dosya boş değilse) listeye ekler.
            if content.strip(): 
                KNOWLEDGE_BASE.append(content)
            else:
                # Boş dosyaları bize bildirir
                print(f"UYARI: {file_path} dosyası boş veya sadece boşluk içeriyor. Bu dosya atlandı.")
    except Exception as e:
        print(f"HATA: {file_path} dosyası okunurken sorun oluştu: {e}")

if not KNOWLEDGE_BASE:
    print("HATA: Hiç dolu .txt dosyası bulunamadı. Lütfen 'data' klasörünüzü kontrol edin.")
else:
    print(f"Başarıyla {len(KNOWLEDGE_BASE)} adet dolu doküman yüklendi.")

# Gömme (Embedding) modelini seçme
EMBEDDING_MODEL = 'gemini-embedding-001' # Gemini için önerilen embedding modeli

def create_embeddings(texts):
    """Metin listesi için gömme (embedding) vektörlerini oluşturur."""
    print("Veri setinin embedding'leri oluşturuluyor...")
    response = client.models.embed_content(
        model=EMBEDDING_MODEL,
        contents=texts,
    )
    embeddings_list = [item.values for item in response.embeddings]
    # Vektörleri numpy dizisine dönüştür
    return np.array(embeddings_list)

# 3. Vektör Veritabanı Oluşturma (Bu örnekte bellek içi bir matris)
knowledge_base_embeddings = create_embeddings(KNOWLEDGE_BASE)
print(f"Oluşturulan embedding sayısı: {len(knowledge_base_embeddings)}")

def retrieve_context(query, top_k=2):
    """Sorgu ile en alakalı veri parçalarını bulur (Geri Alma)."""
    # Sorgunun embedding'ini oluştur
    query_embedding_response = client.models.embed_content(
        model=EMBEDDING_MODEL,
        contents=[query]
    )
    query_embeddings_list = [item.values for item in query_embedding_response.embeddings]
    query_embedding = np.array(query_embeddings_list)

    # Kosinüs benzerliği ile sorgu ve bilgi tabanı vektörleri arasındaki benzerliği hesapla
    # reshape(1, -1) sorgu vektörünün tek satırlı bir matris olmasını sağlar
    similarities = cosine_similarity(query_embedding.reshape(1, -1), knowledge_base_embeddings)

    # En yüksek benzerliğe sahip parçaların indekslerini al
    top_indices = np.argsort(similarities[0])[::-1][:top_k]

    # En alakalı parçaları döndür
    retrieved_chunks = [KNOWLEDGE_BASE[i] for i in top_indices]
    print(f"\n[Geri Alınan Bağlam ({top_k} adet)]:")
    for chunk in retrieved_chunks:
        print(f"- {chunk}")

    return "\n".join(retrieved_chunks)

def generate_response(query, context):
    """Geri alınan bağlamı kullanarak Gemini'den yanıt üretir (Üretim)."""

    # Modelden istediğimiz davranışı belirten prompt şablonu
    prompt = f"""
    Aşağıdaki 'BAĞLAM' bölümünde sağlanan bilgilere dayanarak '{query}' sorusuna yanıt verin.
    Eğer bağlamda yeterli bilgi yoksa, 'Üzgünüm, bu bilgiye sahip değilim.' diye cevap verin.

    BAĞLAM:
    {context} 

    SORU: {query}
    YANIT:
    """

    print("\n[Yanıt Üretiliyor...]")
    response = client.models.generate_content(
        model='gemini-2.0-flash', # Hızlı ve verimli model
        contents=prompt
    )

    return response.text

def simple_rag_pipeline(query):
    """Tüm RAG sürecini çalıştırır."""
    # 4. Geri Alma (Retrieval)
    context = retrieve_context(query)

    # 5. Üretim (Generation)
    final_response = generate_response(query, context)

    print("\n--- Nihai Yanıt ---")
    print(final_response)
    print("--------------------")

# Örnek Sorgu
query1 = "V60 ile kahve demlerken su sıcaklığı kaç derece olmalı?"
simple_rag_pipeline(query1)

print("\n" + "="*50 + "\n")

# Bağlamda olmayan bir sorgu
query2 = "Türkiye'nin başkenti neresidir?"
simple_rag_pipeline(query2)

Dosyalar okunuyor ve filtreleniyor...
Başarıyla 12 adet dolu doküman yüklendi.
Veri setinin embedding'leri oluşturuluyor...
Oluşturulan embedding sayısı: 12

[Geri Alınan Bağlam (2 adet)]:
- HARIO V60
Evde sabah kahvenizi yapmak için harika bir ekipmandır. Kullanımı kolay bir demleme yöntemi olup, sadece birkaç dakikanızı alacak keyifli bir filtre kahve deneyimi sağlar. Konik açıları 60 derece olup bir ‘V’ şekli oluşturur. Bu iki element birleşerek V60 adını almıştır. Bu açılar, hava akışı için yapılmış iç çıkıntılarıyla birleştiğinde, her seferinde tutarlı bir demleme yapmanızı sağlamaya yardımcı olur. Burada dikkat etmeniz gereken püf noktası bir kettle yardımı ile sıcak suyunuzu kontrollü bir şekilde dökmek olacaktır.

Doz: 18 g
Su Miktarı: 300 ml
Su Sıcaklığı: 95C
Demlenme Süresi: 2:30 – 3:00 dakika
Malzemeler
HARIO V60 02
TARTI
KRONOMETRE
DEĞİRMEN
KETTLE
KARAF
HARİO V60 02 KAĞIT FİLTRE
AHŞAP KAŞIK
18 g kahveyi tartarak hazırla ve orta kalınlıkta öğüt.
Filtre kağıdını ekipmana yerl